## Installing libraries

In [1]:
#!pip install node2vec
#!pip install neo4j
#!pip install --upgrade networkx

## Importing Libraries

In [2]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from neo4j import GraphDatabase
from node2vec import Node2Vec
import networkx as nx
import numpy as np
import scipy.sparse as sp
import gensim.downloader as api
from sklearn.preprocessing import StandardScaler

## Connecting to the neo4j database

In [3]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "OLIV00%%"

driver = GraphDatabase.driver(uri, auth=(username, password))

## Loading of the KG

In [4]:
graph = nx.Graph()

with driver.session() as session:
    # Récupérer les nœuds avec leur nom et type
    nodes_query = "MATCH (n) RETURN ID(n) AS id, n.name AS name, n.type AS type"
    nodes_result = session.run(nodes_query)

    # Parcourir les résultats et ajouter les nœuds au graphe
    for record in nodes_result:
        node_id = record["id"]
        node_name = record["name"]
        node_type = record["type"]
        if node_id is not None:
            graph.add_node(node_id, name=node_name, type=node_type)

    # Récupérer les relations
    relations_query = "MATCH ()-[r]->() RETURN ID(startNode(r)) AS source_id, ID(endNode(r)) AS target_id"
    relations_result = session.run(relations_query)

    # Ajouter les relations entre les nœuds
    for record in relations_result:
        source_node_id = record["source_id"]
        target_node_id = record["target_id"]
        if source_node_id is not None and target_node_id is not None:
            graph.add_edge(source_node_id, target_node_id)

# Afficher les informations des nœuds
for node_id, node_data in graph.nodes(data=True):
    print("Node ID:", node_id)
    print("Node Name:", node_data.get("name"))
    print("Node Type:", node_data.get("type"))
    print()

Node ID: 0
Node Name: Iv1
Node Type: fftvInterval

Node ID: 1
Node Name: Iv2
Node Type: fftvInterval

Node ID: 2
Node Name: Iv3
Node Type: fftvInterval

Node ID: 3
Node Name: Iv4
Node Type: fftvInterval

Node ID: 4
Node Name: Iv5
Node Type: fftvInterval

Node ID: 5
Node Name: Iv6
Node Type: fftvInterval

Node ID: 6
Node Name: Iv7
Node Type: fftvInterval

Node ID: 7
Node Name: Ig1
Node Type: fftgInterval

Node ID: 8
Node Name: Ig2
Node Type: fftgInterval

Node ID: 9
Node Name: Ig3
Node Type: fftgInterval

Node ID: 10
Node Name: Ig4
Node Type: fftgInterval

Node ID: 11
Node Name: Ig5
Node Type: fftgInterval

Node ID: 12
Node Name: Ig6
Node Type: fftgInterval

Node ID: 13
Node Name: Ig7
Node Type: fftgInterval

Node ID: 14
Node Name: Static unbalance
Node Type: Explanation

Node ID: 15
Node Name: Couple unbalance
Node Type: Explanation

Node ID: 16
Node Name: Dynamic unbalance
Node Type: Explanation

Node ID: 17
Node Name: Overhung unbalance
Node Type: Explanation

Node ID: 18
Node Name: 

## Features extraction from nodes

In [5]:
# Préparer les caractéristiques des nœuds
node_features = []

# Récupérer les attributs "id", "name" et "type" des nœuds dans le graphe
for node_id in graph.nodes:
    node_data = graph.nodes[node_id]
    node_feature = {
        "id": node_id,
        "name": node_data["name"],
        "type": node_data["type"]
    }
    node_features.append(node_feature)

# Convertir les caractéristiques des nœuds en un tableau numpy
X = np.array([(node["id"], node["name"], node["type"]) for node in node_features])
#X = np.array(node_features)
print(X)

[['0' 'Iv1' 'fftvInterval']
 ['1' 'Iv2' 'fftvInterval']
 ['2' 'Iv3' 'fftvInterval']
 ['3' 'Iv4' 'fftvInterval']
 ['4' 'Iv5' 'fftvInterval']
 ['5' 'Iv6' 'fftvInterval']
 ['6' 'Iv7' 'fftvInterval']
 ['7' 'Ig1' 'fftgInterval']
 ['8' 'Ig2' 'fftgInterval']
 ['9' 'Ig3' 'fftgInterval']
 ['10' 'Ig4' 'fftgInterval']
 ['11' 'Ig5' 'fftgInterval']
 ['12' 'Ig6' 'fftgInterval']
 ['13' 'Ig7' 'fftgInterval']
 ['14' 'Static unbalance' 'Explanation']
 ['15' 'Couple unbalance' 'Explanation']
 ['16' 'Dynamic unbalance' 'Explanation']
 ['17' 'Overhung unbalance' 'Explanation']
 ['18' 'Structural looseness' 'Explanation']
 ['19' 'Angular misalignment' 'Explanation']
 ['20' 'General misalignment' 'Explanation']
 ['21' 'Parallel misalignment' 'Explanation']
 ['22' 'Mechanical loosenes' 'Explanation']
 ['23' 'Component loosenes' 'Explanation']
 ['24' 'Bearing Lubrication Fault' 'Explanation']
 ['25' 'Excessive gear' 'Explanation']
 ['26' 'Excessive loading' 'Explanation']
 ['27' 'Mechanical misalignment' 'Expl

In [6]:
machine_state_ids = [33, 34, 35, 36, 37, 38, 39]
component_ids = [28, 29, 30]

X_comp_state_1 = X[28:31]
X_comp_state_2 = X[33:40]
X_comp_state = np.concatenate((X_comp_state_1, X_comp_state_2), axis = 0)
X_comp_state

array([['28', 'Pump', 'Component'],
       ['29', 'Fan', 'Component'],
       ['30', 'Engine', 'Component'],
       ['33', 'Normal', 'Normal'],
       ['34', 'Imbalance', 'Imbalance'],
       ['35', 'Structural fault', 'Failure'],
       ['36', 'Misalignment', 'Failure'],
       ['37', 'looseness', 'Failure'],
       ['38', 'Bearing Fault', 'Failure'],
       ['39', 'Gear Fault', 'Failure']], dtype='<U25')

## Attention-based Compressed Relational Graph Convolutional Network (ACRGCN)

In [17]:
def ACRGCN(G, X, L, Decoder_layer):
    H = X

    for i in range(L):
        H = RGCN(H, G)
        H = BN(H)
        H = ReLU(H)
        if i < L-1:
            H = Dropout(H)

    H_maxlayer = GAT(H)

    for i in range(Decoder_layer, 0, -1):
        H = DeRGCN(H_maxlayer, G)
        H = BN(H)
        H = ReLU(H)
        if i > 1:
            H = Dropout(H)
        H = H + H_maxlayer

    E_candidate = Sort(ScoreDistMult(H))

    return E_candidate


def RGCN(H, G):
    A = nx.adjacency_matrix(G)  # Use sparse adjacency matrix
    D = sp.diags(A.sum(axis=1).flatten())
    D_inv = sp.linalg.inv(D)
    H = D_inv.dot(A).dot(H)  # R-GCN operation
    return H

def BN(H):
    # Batch Normalization operation
    scaler = StandardScaler()
    H = scaler.fit_transform(H)
    return H

def ReLU(H):
    # ReLU activation function
    return np.maximum(H, 0)

def Dropout(H, p=0.5):
    # Dropout operation
    mask = np.random.binomial(1, p, size=H.shape)
    H = H * mask / p
    return H

def GAT(H):
    # GAT operation
    return H

def DeRGCN(H, G):
    A = nx.adjacency_matrix(G)  # Use sparse adjacency matrix
    D = sp.diags(A.sum(axis=1).flatten())
    D_inv = sp.linalg.inv(D)
    H = D_inv.dot(A).dot(H)  # DeR-GCN operation
    return H

def ScoreDistMult(H):
    # Score calculation using DistMult
    scores = np.dot(H, np.transpose(H))
    return scores

def Sort(scores):
    # Sort the scores in descending order
    sorted_scores = sorted(scores, reverse=True)
    return sorted_scores


## Model execution

In [18]:
# Appliquer l'algorithme ACRGCN
L = 3  # Encoder layer number
Decoder_layer = 2  # Decoder layer number
E_candidates = ACRGCN(graph, X, L, Decoder_layer)

# Afficher les résultats
print(E_candidates)


C:\Users\olivi\anaconda3\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\olivi\anaconda3\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


TypeError: no supported conversion for types: (dtype('float64'), dtype('<U'))

## Close session and driver at the end

In [40]:
session.close()
driver.close()